# Convert to parquet

In [ ]:
import pandas as pd

In [ ]:
# def_df = pd.read_csv("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/defaulter_data_6k_ids.csv", index_col=[0])
# pay_df = pd.read_csv("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/payer_data_20k_ids.csv", index_col=[0])



In [ ]:
# def_df = pd.read_csv("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/defaulter_data_20k_ids.csv", index_col=[0])



In [ ]:
def compress_datatypes(df, **kwargs):
    """
    Reduces size of dataframe by downcasting numerical columns
    """
    input_size = df.memory_usage(index=True).sum()/ 1024
    print("new dataframe size: ", round(input_size,2), 'kB')
    
    in_size = df.memory_usage(index=True).sum()
    for type in ["float", "integer"]:
        l_cols = list(df.select_dtypes(include=type))
        for col in l_cols:
            df[col] = pd.to_numeric(df[col], downcast=type)
    out_size = df.memory_usage(index=True).sum()
    ratio = (1 - round(out_size / in_size, 2)) * 100
    
    print("optimized size by {} %".format(round(ratio,2)))
    print("new dataframe size: ", round(out_size / 1024,2), " kB")
    
    return df

def_df_compress = compress_datatypes(def_df)
# pay_df_compress = compress_datatypes(pay_df)

# convert to parquet:
def_df_compress.to_parquet('/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/defaulter_data_20k_ids_compress.parquet')
print('\nparquet defaulter done')
# pay_df_compress.to_parquet('/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/payer_data_20k_ids.parquet')
print('\nparquet payer done')


# Reduce dataset size  w Kaggle

In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyarrow as pa
import pyarrow.parquet as pq
import time
from tqdm.auto import tqdm
from tqdm.notebook import trange, tqdm
from time import sleep


In [13]:
test_path = '/Users/sjoerddewit/Downloads/test_data.csv'
parquet_file = '/Users/sjoerddewit/Downloads/test_data.parquet'
test = pd.read_csv(test_path, nrows=10000)

In [ ]:
# train_path = '/Users/sjoerddewit/Downloads/train_data.csv'
# parquet_file = '/Users/sjoerddewit/Downloads/train_data.parquet'
# train = pd.read_csv(train_path, nrows=10000)

In [14]:
def get_col_dtypes(data):
    col_dtypes = {}
    for col in data.columns:
        if str(data[col].dtype) == 'float64':
            col_dtypes[col] = 'float32'
        elif str(data[col].dtype) == 'int64':
            col_dtypes[col] = 'int32'
        elif str(data[col].dtype) == 'object':
            col_dtypes[col] = 'category'
        else:
            col_dtypes[col] = str(data[col].dtype)
    return col_dtypes

In [15]:
col_dtypes = get_col_dtypes(test)

In [16]:
def write_parquet(path, save_loc, col_dtypes, chunk_size=10000):
    schema = ''
    writer = ''
    for i,chunk in enumerate(pd.read_csv(path, dtype=col_dtypes, iterator=True, chunksize=10000)):
        if i == 0:
            schema = pa.Table.from_pandas(df=chunk).schema
            writer = pq.ParquetWriter(save_loc, schema, compression='snappy')
        table = pa.Table.from_pandas(chunk, schema=schema)
        writer.write_table(table)
    return None

In [17]:
start = time.time()
write_parquet(test_path, parquet_file, col_dtypes)
end = time.time()
minutes, seconds = divmod(end-start, 60)
print(f"Time taken for converting csv to parquet: {minutes} minutes and {seconds} seconds....")


Time taken for converting csv to parquet: 6.0 minutes and 56.42820596694946 seconds....


# Chuncking the parquet test file

In [1]:
import dask.dataframe as dd

path = "/Users/sjoerddewit/Downloads/"

ddf = dd.read_parquet('/Users/sjoerddewit/Downloads/test_data.parquet')
ddf.repartition(8).to_parquet(path,
                             allow_truncated_timestamps=True)


ValueError: Failed to convert partition to expected pyarrow schema:
    `ArrowInvalid('Integer value 462378 not in range: -128 to 127', 'Conversion failed for column customer_ID with type category')`

Expected partition schema:
    customer_ID: dictionary<values=string, indices=int8, ordered=0>
    S_2: dictionary<values=string, indices=int8, ordered=0>
    P_2: float
    D_39: float
    B_1: float
    B_2: float
    R_1: float
    S_3: float
    D_41: float
    B_3: float
    D_42: float
    D_43: float
    D_44: float
    B_4: float
    D_45: float
    B_5: float
    R_2: float
    D_46: float
    D_47: float
    D_48: float
    D_49: float
    B_6: float
    B_7: float
    B_8: float
    D_50: float
    D_51: float
    B_9: float
    R_3: float
    D_52: float
    P_3: float
    B_10: float
    D_53: float
    S_5: float
    B_11: float
    S_6: float
    D_54: float
    R_4: float
    S_7: float
    B_12: float
    S_8: float
    D_55: float
    D_56: float
    B_13: float
    R_5: